# Optuna study

> Combine it with papermill and wandb for seamless hyperparameter tuning

In [9]:
import sys
sys.path.append('..')
import os
import optuna
from tsai.optuna import *
from tsai.basics import load_object
import papermill as pm
from tsai.optuna import run_optuna_study
from fastcore.basics import *
from optuna.distributions import *
from optuna.samplers import TPESampler
import wandb

In [10]:
config = AttrDict(
    study_name = 'general_study_extended', # name of the Optuna study
    study_type = 'bayesian', # 'bayesian' or 'gridsearch' or 'random'
    n_trials = 50, # number of trials
    train_nb = f'{os.getcwd()}/solfsmy_train.ipynb', # path to the notebook to be executed
    search_space = {
        "arch.attn_dropout": DiscreteUniformDistribution(0.0, 0.5, 0.1),
        "arch.d_model": IntUniformDistribution(32, 512, 32),
        "arch.d_ff": IntUniformDistribution(32, 512, 32),
        "arch.decomposition": CategoricalDistribution([True, False]),
        "arch.dropout": DiscreteUniformDistribution(0.0, 0.5, 0.1), 
        "arch.individual": CategoricalDistribution([True, False]),
        "arch.n_layers": IntUniformDistribution(1, 6, 1),
        "arch.n_heads": CategoricalDistribution([2, 4, 8, 16, 32]),
        "init_weights": CategoricalDistribution([True, False]), # true = kaiming
        "lookback": CategoricalDistribution([18, 24, 36, 128, 192]),
        "deltaHL": FloatDistribution(1., 15., step=1.),
    },
    # Add extra parameters that are fixed, but not part of the search space
    extra_params = {
        "n_epoch": 100,
        "bs": 32
    },
    use_wandb = True,
    wandb_mode = 'online'
)

config

```json
{ 'extra_params': {'bs': 32, 'n_epoch': 100},
  'n_trials': 50,
  'search_space': { 'arch.attn_dropout': DiscreteUniformDistribution(high=0.5, low=0.0, q=0.1),
                    'arch.d_ff': IntUniformDistribution(high=512, low=32, step=32),
                    'arch.d_model': IntUniformDistribution(high=512, low=32, step=32),
                    'arch.decomposition': CategoricalDistribution(choices=(True, False)),
                    'arch.dropout': DiscreteUniformDistribution(high=0.5, low=0.0, q=0.1),
                    'arch.individual': CategoricalDistribution(choices=(True, False)),
                    'arch.n_heads': CategoricalDistribution(choices=(2, 4, 8, 16, 32)),
                    'arch.n_layers': IntUniformDistribution(high=6, low=1, step=1),
                    'deltaHL': FloatDistribution(high=15.0, log=False, low=1.0, step=1.0),
                    'init_weights': CategoricalDistribution(choices=(True, False)),
                    'lookback': CategoricalDistribution(choices=(18, 24, 36, 128, 192))},
  'study_name': 'general_study_extended',
  'study_type': 'bayesian',
  'train_nb': '/workspaces/sw-driver-forecaster/dev_nbs/solfsmy_train.ipynb',
  'use_wandb': True,
  'wandb_mode': 'online'}
```

In [11]:
def create_objective(train_nb, search_space, extra_params=None, use_wandb=False):
    """
        Create objective function to be minimized by Optuna.
        Inputs:
            trial: Optuna trial object
            train_nb: path to the training notebook
            search_vars: keys of the search space to be used
            wandb_group: name of the wandb group to be used
        Output:
            valid_loss: validation loss
    """
    def objective(trial:optuna.Trial):
        # Define the parameters to be passed to the training notebook through papermill
        pm_parameters = {}
        for k,v in search_space.items():
            pm_parameters['config.' + k] = trial._suggest(k, v)

        # Add the extra parameters to the dictionary. The key of every parameter 
        # must be 'config.<param_name>'
        if extra_params is not None:
            for k,v in extra_params.items():
                pm_parameters['config.' + k] = v
                
        # If using wandb, enable that in the training runs, all of them gathered
        # into a group (NOTE: The train nb must have and use these config arguments)
        if use_wandb:
            pm_parameters['config.use_wandb'] = True
            pm_parameters['config.wandb_group'] = config.study_name + '_runs'

        # Call the training notebook using papermill (don't print the output)
        stdout_file = open('tmp/pm_stdout.txt', 'w')
        stderr_file = open('tmp/pm_stderr.txt', 'w')

        pm.execute_notebook(
            train_nb,
            './tmp/pm_output.ipynb',
            parameters = pm_parameters,
            stdout_file = stdout_file,
            stderr_file = stderr_file
        )

        # Close the output files
        stdout_file.close()
        stderr_file.close()

        # Get the output value of interest from the source notebook
        loss = None
        %store -r valid_loss
        return valid_loss

    return objective

In [12]:
obj = create_objective(config.train_nb, config.search_space, 
                       extra_params=config.extra_params, use_wandb=True)
study = run_optuna_study(obj, study_type='bayesian', direction='minimize', path='./tmp',
                 study_name=config.study_name, n_trials=config.n_trials)

[I 2024-07-10 08:59:39,758] A new study created in memory with name: general_study_extended
  0%|          | 0/50 [00:00<?, ?it/s]Passed unknown parameter: config.arch.attn_dropout
Passed unknown parameter: config.arch.d_model
Passed unknown parameter: config.arch.d_ff
Passed unknown parameter: config.arch.decomposition
Passed unknown parameter: config.arch.dropout
Passed unknown parameter: config.arch.individual
Passed unknown parameter: config.arch.n_layers
Passed unknown parameter: config.arch.n_heads
Passed unknown parameter: config.init_weights
Passed unknown parameter: config.lookback
Passed unknown parameter: config.deltaHL
Passed unknown parameter: config.n_epoch
Passed unknown parameter: config.bs
Passed unknown parameter: config.use_wandb
Passed unknown parameter: config.wandb_group
Best trial: 0. Best value: 2.58123:   2%|▏         | 1/50 [01:03<52:12, 63.93s/it]Passed unknown parameter: config.arch.attn_dropout
Passed unknown parameter: config.arch.d_model
Passed unknown pa

[I 2024-07-10 09:00:43,692] Trial 0 finished with value: 2.5812342166900635 and parameters: {'arch.attn_dropout': 0.1, 'arch.d_model': 64, 'arch.d_ff': 256, 'arch.decomposition': False, 'arch.dropout': 0.30000000000000004, 'arch.individual': False, 'arch.n_layers': 2, 'arch.n_heads': 16, 'init_weights': True, 'lookback': 24, 'deltaHL': 8.0}. Best is trial 0 with value: 2.5812342166900635.


Best trial: 0. Best value: 2.58123:   4%|▍         | 2/50 [02:23<58:38, 73.31s/it]Passed unknown parameter: config.arch.attn_dropout
Passed unknown parameter: config.arch.d_model
Passed unknown parameter: config.arch.d_ff
Passed unknown parameter: config.arch.decomposition
Passed unknown parameter: config.arch.dropout
Passed unknown parameter: config.arch.individual
Passed unknown parameter: config.arch.n_layers
Passed unknown parameter: config.arch.n_heads
Passed unknown parameter: config.init_weights
Passed unknown parameter: config.lookback
Passed unknown parameter: config.deltaHL
Passed unknown parameter: config.n_epoch
Passed unknown parameter: config.bs
Passed unknown parameter: config.use_wandb
Passed unknown parameter: config.wandb_group


[I 2024-07-10 09:02:03,565] Trial 1 finished with value: 5.956546783447266 and parameters: {'arch.attn_dropout': 0.1, 'arch.d_model': 352, 'arch.d_ff': 256, 'arch.decomposition': False, 'arch.dropout': 0.1, 'arch.individual': False, 'arch.n_layers': 4, 'arch.n_heads': 16, 'init_weights': True, 'lookback': 192, 'deltaHL': 10.0}. Best is trial 0 with value: 2.5812342166900635.


Best trial: 0. Best value: 2.58123:   6%|▌         | 3/50 [04:02<1:06:40, 85.12s/it]Passed unknown parameter: config.arch.attn_dropout
Passed unknown parameter: config.arch.d_model
Passed unknown parameter: config.arch.d_ff
Passed unknown parameter: config.arch.decomposition
Passed unknown parameter: config.arch.dropout
Passed unknown parameter: config.arch.individual
Passed unknown parameter: config.arch.n_layers
Passed unknown parameter: config.arch.n_heads
Passed unknown parameter: config.init_weights
Passed unknown parameter: config.lookback
Passed unknown parameter: config.deltaHL
Passed unknown parameter: config.n_epoch
Passed unknown parameter: config.bs
Passed unknown parameter: config.use_wandb
Passed unknown parameter: config.wandb_group


...

Best trial: 35. Best value: 2.28338:  98%|█████████▊| 49/50 [59:05<01:15, 75.13s/it]Passed unknown parameter: config.arch.attn_dropout
Passed unknown parameter: config.arch.d_model
Passed unknown parameter: config.arch.d_ff
Passed unknown parameter: config.arch.decomposition
Passed unknown parameter: config.arch.dropout
Passed unknown parameter: config.arch.individual
Passed unknown parameter: config.arch.n_layers
Passed unknown parameter: config.arch.n_heads
Passed unknown parameter: config.init_weights
Passed unknown parameter: config.lookback
Passed unknown parameter: config.deltaHL
Passed unknown parameter: config.n_epoch
Passed unknown parameter: config.bs
Passed unknown parameter: config.use_wandb
Passed unknown parameter: config.wandb_group


[I 2024-07-10 09:58:45,044] Trial 48 finished with value: 2.3322994709014893 and parameters: {'arch.attn_dropout': 0.4, 'arch.d_model': 288, 'arch.d_ff': 32, 'arch.decomposition': False, 'arch.dropout': 0.0, 'arch.individual': True, 'arch.n_layers': 3, 'arch.n_heads': 8, 'init_weights': False, 'lookback': 24, 'deltaHL': 2.0}. Best is trial 35 with value: 2.2833826541900635.


Best trial: 35. Best value: 2.28338: 100%|██████████| 50/50 [1:00:38<00:00, 72.78s/it]

[I 2024-07-10 10:00:18,674] Trial 49 finished with value: 2.772886276245117 and parameters: {'arch.attn_dropout': 0.2, 'arch.d_model': 448, 'arch.d_ff': 128, 'arch.decomposition': True, 'arch.dropout': 0.2, 'arch.individual': False, 'arch.n_layers': 4, 'arch.n_heads': 2, 'init_weights': False, 'lookback': 24, 'deltaHL': 4.0}. Best is trial 35 with value: 2.2833826541900635.

Optuna study saved to tmp/general_study_extended.pkl
To reload the study run: study = joblib.load('tmp/general_study_extended.pkl')



Study statistics    : 
  Study name        : general_study_extended
  # finished trials : 50
  # pruned trials   : 0
  # complete trials : 50

Best trial          :
  value             : 2.2833826541900635
  best_params = {'arch.attn_dropout': 0.2, 'arch.d_model': 224, 'arch.d_ff': 224, 'arch.decomposition': False, 'arch.dropout': 0.0, 'arch.individual': True, 'arch.n_layers': 2, 'arch.n_heads': 8, 'init_weights': False, 'lookback': 24, 'deltaHL': 2.0}



In [ ]:
run = wandb.init(config=config, mode=config.wandb_mode, 
                 job_type='optuna-study') if config.use_wandb else None

In [ ]:
if run is not None:
    run.log(dict(study.best_params, **{'best_value': study.best_value, 
                                       'best_trial_number': study.best_trial.number}))
    run.log_artifact(f'./tmp/{config.study_name}.pkl', type='optuna_study')
    run.log({
        'contour': optuna.visualization.plot_contour(study),
        'edf': optuna.visualization.plot_edf(study),
        'intermediate_values': optuna.visualization.plot_intermediate_values(study),
        'optimization_history': optuna.visualization.plot_optimization_history(study),
        'parallel_coordinate' : optuna.visualization.plot_parallel_coordinate(study),
        'param_importances': optuna.visualization.plot_param_importances(study),
        'slice': optuna.visualization.plot_slice(study)
    })

In [28]:
if run is not None:
    run.finish()